In [30]:
import pandas as pd
import numpy as np
from os import path as os_path, getcwd
from sys import path as sys_path
import matplotlib.pyplot as plt

import time

import gdax

import datetime

# Importing Keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, GRU, LSTM
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.layers import ELU
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [31]:
# Get data every 15min between 2 dates
public_client = gdax.PublicClient()
dt = datetime.timedelta(minutes=2*60+15)
begin = datetime.datetime(2017, 1, 1, 0, 0) # Change begin date (YYYY/DD/MM/h/min)
end = datetime.datetime(2017, 1, 2, 0, 0)   # Change end date
data=public_client.get_product_historic_rates('ETH-USD', start = begin, end = begin + dt, granularity=900)


current_time = begin
df_year = pd.DataFrame()
while(current_time < end):
    data=public_client.get_product_historic_rates('ETH-USD', start = current_time, end = current_time + dt, granularity=900)
    df = pd.DataFrame(data, columns=['time','low','high','open', 'close', 'volume'])
    df.time = pd.to_datetime(df['time'], unit='s')
    current_time = df.time[0]
    df=df.iloc[::-1].reset_index(drop=True)
    df_year = df_year.append(df)
    time.sleep(0.5)
    print(current_time)
df_year = df_year.reset_index(drop=True)


2017-01-01 02:00:00
2017-01-01 04:00:00
2017-01-01 06:00:00
2017-01-01 08:00:00
2017-01-01 10:00:00
2017-01-01 12:00:00
2017-01-01 14:00:00
2017-01-01 16:00:00
2017-01-01 18:00:00
2017-01-01 20:00:00
2017-01-01 22:00:00
2017-01-02 00:00:00


In [32]:
display(df_year)

    

,time,low,high,open,close,volume
0,2017-01-01 00:00:00,8.11,8.16,8.16,8.16,378.420945
1,2017-01-01 00:15:00,8.16,8.24,8.16,8.24,252.619480
2,2017-01-01 00:30:00,8.24,8.27,8.24,8.27,237.637246
3,2017-01-01 00:45:00,8.25,8.27,8.27,8.26,109.079011
4,2017-01-01 01:00:00,8.26,8.29,8.26,8.29,270.898827
5,2017-01-01 01:15:00,8.27,8.29,8.27,8.29,280.979249
6,2017-01-01 01:30:00,8.27,8.29,8.29,8.28,84.835125
7,2017-01-01 01:45:00,8.28,8.29,8.28,8.29,393.862555
8,2017-01-01 02:00:00,8.30,8.30,8.30,8.30,11.993290
9,2017-01-01 02:00:00,8.30,8.30,8.30,8.30,11.993290
